# Debiasing Word Embedding 

## 0. make word embeddings 

In [1]:
import pandas as pd
import re
import tqdm as notebook_tqdm
import torch
import torch.nn as nn
from gensim import models

import seaborn as sns

c:\users\이육샛별\appdata\local\programs\python\python37\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2")

In [3]:
df = pd.read_csv('dataset/dataset.csv', encoding='UTF-8')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1314 entries, 0 to 1313
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1314 non-null   int64 
 1   동화          1275 non-null   object
dtypes: int64(1), object(1)
memory usage: 20.7+ KB


In [4]:
df['동화'][4]

"6월 6일은 나라를 지키기 위해 목숨을 바친 모든 이들의 넋을 기리는 법정 기념일 현충일이랍니다.  병자호란 임진왜란을 거치고 또 35년간 일제 강점기를 거치고 또 한국전쟁으로 남과 북이 갈라지고. 이렇게 슬프고 아픈 역사 속에서 우리나라를 지키기 위해 싸우다가 목숨을 잃은 분들이 너무도 많답니다. 우리 모두가 6월 6일 현충일을 맞아서 나라를 위해 목숨을 바친 그리고 나라를 위해 열심히 이름을 빚는 분들에게 더더욱 감사하고 자랑스러운 마음을 가져봤으면 좋겠어요.  그분들이 있기에 지금 우리가 이렇게 편하게 지낼 수 있는 것일 테니까요.  오늘 신나는 동화여행에서는 현충일을 맞아서 나라를 빼앗겼던 일제 강점기 그 시대에 나라를 잃은 슬픔이 얼마나 커다란 것인지 느끼게 해준 세상에서 제일 슬픈 금메달 이야기를 해볼까요.  그럼 신나는 동화 여행에서 달리기 소년 손기정을 만나러 가볼까요.  우리 할아버지입니다. 어려서부터 나는 할아버지와 한 방을 썼어요. 할아버지는 늘 신문을 오려 붙이고 우표책을 정리하셨죠.  할아버지 헌 우표는 뭐하러 모으세요.  신문 오리는 거 재미있으세요. 할아버지는 부지런하고 꼼꼼하셨어요.  할아버지와 저는 사이가 아주 좋았답니다.  에게 스케이트를 사주시고 자전거 타는 법도 가르쳐 주신 게 할아버지거든요. 넘어지는 걸 겁내지 마라 무릎이 까지고 피도 나봐야 잘 달릴 수 있는 거야 할아버지는 늘 이렇게 말씀하셨어요.  몸이 건강해야 마음도 건강해진다고요 운동회 날이면 사람들 속에 할아버지가 늘 서 계셨어요.  할아버지가 지켜봐 주시니까 저는 너무나도 힘이 났죠. 니 앞에 가는 사람은 너만큼 힘들고 니 뒤에 따라오는 사람은 너보다 더 힘들단다 할아버지 말씀 때문에 나는 힘든 줄도 모르고 달릴 수 있었죠.  골목에서 달리기 시합을 하던 날이었어요.  달리기에서 1등을 한 동네 형이 자랑스럽게 말했죠. 이러면 나도 올림픽 마라톤에서 금메달을 딸 거야 옛날에 손기정이라는 사람이 올림픽 금메달을 땄는데 일본 놈들이 자기네 금메달이라고 했대 그래서

In [5]:
for i in range(len(df['동화'])):
    text = str(df['동화'][i])
    full_text = ''.join(text)
    
full_text

'하루는 노인과 그의 아내가 누추한 집 앞에 앉아 잠시 일손을 놓고 휴식들을 취하고 있는데, 난데없이 검정 말 네 마리가 끄는 화려한 마차가 그들 쪽으로 달려오더니 부자차림의 사내 한 명이 내리는 거예요. 농부가 일어나 그 권세가에게 다가가 “뭐 필요하신 거라고 있으신지요?”라고 물었어요. 손님이 노인에게 손을 내밀더니 말했어요. “바라는 건 없습니다, 다만 시골 밥상 한 번 먹어보고 싶군요. 늘 즐겨먹으시던 그 감자 요리 말입니다. 준비해주시겠다면 즐거이 먹겠습니다.” 농부는 미소를 보이더니 말했어요. “네, 아무래도 당신은 백작이거나 왕자님 아니시면 공작이신 모양이군요. 귀족 분들이 종종 그런 음식상을 바라실 때가 있지요, 해드리리다.” 아내가 부엌으로 가 감자를 씻어 갈기 시작하더니 나중엔 둥글게 말아서 시골사람들이 즐겨 먹는 걸 만들었지요. 아내가 바삐 요리를 하는 동안, 농부가 손님에게 말했어요. “잠시 정원으로 드시지요, 제가 아직 해둘 일이 좀 남아서요.” 그는 정원에 구멍 몇 개를 이미 파놓은 상태였던지라, 거기에 나무 몇 그루를 심으려던 참이었지요. “자녀가 없으시군요,”라며 손님이 말했어요. “도와줄 만한 이들은 없는 겁니까?” “네,”라며 농부가 대답했어요. “사실, 아들놈이 한 명 있긴 있었는데, 오래전에 세상으로 나갔죠. 변변치 못한 놈이 머리가 좋고 이해가 빠르다 보니 배우는 게 고작 나쁜 짓들뿐이더군요. 그러다 결국 집을 나가버렸죠, 그 이후론 그 애 소식을 듣지 못했습니다.” 노인은 어린 나무 한 그루를 집더니 땅에 파놓은 구멍 속에다 넣고 그 옆에 말뚝 하나를 때려 박았지요. 그런 다음 흙을 파 넣고 단단히 밟아 주었답니다. 땅 위로 나온 나무줄기를 맨 위와 맨 아래 그리고 중간 이렇게 세 부분을 노끈으로 말뚝과 단단히 묶었어요. “말해주십시오,”라며 손님이 말했어요. “왜 저기 저 구부러지고 울퉁불퉁한 나무는 묶지 않으신 겝니까, 저 구석에 있는 거 말입니다, 아래로 구부러져 거의 땅에 달라붙은 있는 거요, 저 나무도 말뚝을 밖

In [61]:
len(full_text)

8675

In [58]:
print(full_text)

0        우리 친구들은 미술이 뭐라고 생각하나요.  종이에 그림을 그리는 것 아니면 멋진 ...1       2022년 5월 5일은 방정환 선생님이 어린이날을 만든 지 100주년이 되는 날이랍...2       5월은 가정의 달이라고 할 만큼 가족들의 소중함을 생각해보게 하는 기념일이 참 많은...3        우리 친구들은 또래 친구들을 만나면 보통 어떤 놀이를 하면서 시간을 보내나요?  ...4       6월 6일은 나라를 지키기 위해 목숨을 바친 모든 이들의 넋을 기리는 법정 기념일 ...                              ...                        1309    옛날에 한 번은 마부가 포도주를 잔득 실은 ‘달구지’를 끌고 가다 그만 속도를 너무...1310    어느 커다란 도시에 나이 드신 어머님 한 분이 자신의 방에 저녁 내내 앉아 그동안 ...1311    가난한 시골 소년이 어느 날 교회에서 목사님이 “하늘나라 왕국에 들어가길 원하는 자...1312    어느 날 오후에 아기 예수가 아기침대에 누워 잠을 자고 있었답니다. 그때 그의 어머...1313    하루는 노인과 그의 아내가 누추한 집 앞에 앉아 잠시 일손을 놓고 휴식들을 취하고 ...Name: 동화, Length: 1314, dtype: object


In [68]:
from kor2vec import Kor2Vec

kor2vec = Kor2Vec(embed_size=128)

kor2vec.train("dataset/dataset.csv", "model.kor2vec", batch_size=200)

Reading Corpus lines


Corpus Sampling: 100%|████████████████████████████████████████████████████████████| 1684/1684 [00:04<00:00, 411.27it/s]


Training kor2vec
Loading Word_sample corpus
Loading corpus finished
CUDA Available/count: False 0
training on  cpu


EP 0: 100%|████████████████████████████████████████████████████████████████████████| 3512/3512 [06:09<00:00,  9.51it/s]


{'epoch': 0, 'train_ep_loss': 1.3606191934478853}


...

{'epoch': 7, 'train_ep_loss': 1.197156763361907}


EP 8: 100%|████████████████████████████████████████████████████████████████████████| 3512/3512 [06:27<00:00,  9.07it/s]


{'epoch': 8, 'train_ep_loss': 1.1876988652626854}


EP 9: 100%|████████████████████████████████████████████████████████████████████████| 3512/3512 [06:30<00:00,  9.00it/s]


{'epoch': 9, 'train_ep_loss': 1.178092176930508}


In [ ]:
kor2vec.save("embedding_model")

## 1. Bias Evaluation 

In [6]:
from kor2vec import Kor2Vec
kor2vec = Kor2Vec.load("embedding_model")

kor2vec.embedding("옛날옛날에 커다란 호랑이가 살고 있었어요.")

tensor([[-5.9792e-01,  4.6852e-01, -6.2760e-01,  8.0020e-01, -9.0955e-01,
         -5.9727e-01, -8.2120e-01, -5.7056e-01, -2.5505e-01, -5.9409e-01,
          8.8992e-01,  1.4842e+00, -2.9688e-01, -4.2640e-01, -5.9283e-01,
         -1.4650e-01,  5.2687e-01,  9.0604e-01,  1.4035e+00, -5.7625e-01,
         -3.7729e-01,  9.6969e-01,  8.2274e-01, -1.1566e+00,  6.7213e-02,
         -8.3325e-01, -7.8475e-01,  2.1273e-01,  5.6653e-01, -1.4982e-01,
          3.1909e-01, -7.3094e-01,  7.1949e-01,  5.6684e-01,  1.1343e+00,
         -7.4837e-01, -5.5475e-01,  1.0884e+00, -1.5542e-01, -3.8770e-01,
         -4.9190e-01,  1.2128e-01, -1.0822e-01,  1.2499e-01,  1.0427e-02,
          1.4682e+00,  9.9037e-01, -3.1604e-01,  1.3287e-01, -1.3185e+00,
         -1.8763e-02, -5.9029e-01,  5.2883e-01,  3.7427e-01,  7.9970e-01,
         -5.8155e-02,  4.4877e-01, -1.2376e-03, -8.2393e-01,  2.2272e-01,
          7.7622e-01,  9.8738e-01, -4.3114e-01, -4.9727e-02, -3.9917e-01,
          1.0586e+00,  1.1407e+00,  5.

In [46]:
#cosine similiarity
cos = torch.nn.CosineSimilarity(dim=1)

male_keyword = "나무꾼"
female_keyword = "신데렐라"
search_keyword = "-"+"혼인"

output = [cos(kor2vec.embedding(male_keyword), kor2vec.embedding(search_keyword)), cos(kor2vec.embedding(female_keyword), kor2vec.embedding(search_keyword))]

male_similarity = str(str(output[0]).split('[')[1]).split(']')[0]
female_similarity = str(str(output[1]).split('[')[1]).split(']')[0]

print(male_keyword, search_keyword + ':', float(male_similarity))
print(female_keyword, search_keyword + ':', float(female_similarity))

나무꾼 -혼인: 0.1315
신데렐라 -혼인: 0.0167


# 3. Define gender direction

In [122]:
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA

In [132]:
def doPCA(a,b,num_components):
    matrix = []
    center = (kor2vec.embedding(a) + kor2vec.embedding(b))/2

    matrix.append((kor2vec.embedding(a) - center).detach().numpy())
    matrix.append((kor2vec.embedding(b) - center).detach().numpy())
    matrix = np.array(matrix).reshape(2,128)
    
    scaler = MinMaxScaler()
    data_rescaled = scaler.fit_transform(matrix)
    
    pca = PCA(n_components = num_components)
    pca = pca.fit_transform(data_rescaled)
    
    return pca

In [164]:
new_direction = doPCA('아빠','엄마',0.99)
new_direction

array([[ 5.024938 ],
       [-5.0249324]], dtype=float32)

In [165]:
norm_direction = new_direction/ np.linalg.norm(new_direction, axis=1)[:, np.newaxis]
norm_direction

array([[ 1.],
       [-1.]], dtype=float32)

In [ ]:
def diff(self, word1, word2):
        v = self.vecs[self.index[word1]] - self.vecs[self.index[word2]]
        return v/np.linalg.norm(v)

In [ ]:
words = sorted([w for w in model.vocab], key=lambda w: model.vocab[w].index)
vecs = [model[w] for w in words]

In [145]:
words = sorted([w for w in kor2vec.vocab], key=lambda w: kor2vec.vocab[w].index)

TypeError: 'WordCharVocab' object is not iterable

In [ ]:
def diff(self, word1, word2):
        v = self.vecs[self.index[word1]] - self.vecs[self.index[word2]]
        return v/np.linalg.norm(v)

In [151]:
v = kor2vec.embedding('아빠').detach().numpy() - kor2vec.embedding('엄마').detach().numpy()
v/np.linalg.norm(v)

array([[ 0.        ,  0.        ,  0.        ,  0.04878752,  0.01974712,
         0.        ,  0.        ,  0.        ,  0.        ,  0.07853492,
         0.        ,  0.        ,  0.08417387, -0.16987786,  0.        ,
         0.0554847 , -0.03808837,  0.        ,  0.        , -0.18545742,
        -0.01373033, -0.08824214,  0.00779419,  0.        ,  0.        ,
         0.        ,  0.        ,  0.00864821,  0.        ,  0.        ,
         0.        , -0.07741772, -0.0477547 ,  0.        ,  0.26623616,
         0.11537194, -0.09130674, -0.04960033,  0.01190929, -0.06102362,
        -0.06741846,  0.03971662, -0.02090386,  0.        ,  0.        ,
         0.        , -0.1211992 , -0.00250281,  0.00821899,  0.08653627,
        -0.00829001, -0.09112462,  0.05969181,  0.        ,  0.11385895,
         0.        ,  0.02652791,  0.10450457,  0.        , -0.02511589,
         0.11587538,  0.        , -0.17142195,  0.10516644,  0.18741913,
         0.01184638, -0.01188518,  0.069377  ,  0.0

In [152]:
def remove_directions(self, directions): #directions better be orthogonal
        self.desc += ", removed"
        for direction in directions:
            self.desc += " "
            if type(direction) is np.ndarray:
                v = direction / np.linalg.norm(direction)
                self.desc += "vector "
            else:
                w1, w2 = direction
                v = self.diff(w1, w2)
                self.desc += w1 + "-" + w2
            self.vecs = self.vecs - self.vecs.dot(v)[:, np.newaxis].dot(v[np.newaxis, :])
        self.normalize()